In [28]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

In [2]:
!gdown --id 1PY0cG1u96UAKY563Gt8bzA3tuTA4SRyN

zsh:1: command not found: gdown


In [3]:
! unzip spotify_dataset.zip

unzip:  cannot find or open spotify_dataset.zip, spotify_dataset.zip.zip or spotify_dataset.zip.ZIP.


In [4]:
# load the data into a dataframe
df = pd.read_csv("dataset.csv", index_col=[0])

In [5]:
# use the shape property to find (rows, columns)
print('There are {} rows and {} columns.'.format(df.shape[0], df.shape[1]))

There are 114000 rows and 20 columns.


In [6]:
#view samples of the data
df.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


In [7]:
# use dtypes prooperty to find the data type of each column
print(df.dtypes)

track_id             object
artists              object
album_name           object
track_name           object
popularity            int64
duration_ms           int64
explicit               bool
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
track_genre          object
dtype: object


We can see there are a couple columns which the data type should be changed.

- track_id should be a integer
- artists should be string
- album_name should be string
- track_name should be string
- track_genre should be string

In [8]:
# find the amount of missing data in each column
print(df.isnull().sum())

track_id            0
artists             1
album_name          1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64


In [9]:
#count the frequency of missing values in each row (i.e. how many columns are empty)
df.isnull().sum(axis=1).sort_values(ascending = False)

65900     3
0         0
75997     0
76008     0
76007     0
         ..
37995     0
37994     0
37993     0
37992     0
113999    0
Length: 114000, dtype: int64

Out of 114000 rows, only one row have missing data for "artists", "album_name", and "track_name", so we can simply drop that row.

In [10]:
# delete the row with missing data
df = df.dropna()

In [11]:
#remove any duplicate rows (each of the row must be unique - duplication is not allowed)
df.drop_duplicates(inplace=True)

In [12]:
# check for invalid data
df.describe(include='all')

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
count,113549,113549,113549,113549,113549.000000,1.135490e+05,113549,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549
unique,89740,31437,46589,73608,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114
top,6S3JlDAGk3uu3NtZbPnuhS,The Beatles,Alternative Christmas 2022,Run Rudolph Run,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,acoustic
freq,9,279,195,151,NaN,NaN,103831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000
mean,NaN,NaN,NaN,NaN,33.324433,2.280814e+05,NaN,0.567031,0.642091,5.309452,-8.243408,0.637866,0.084674,0.314064,0.155703,0.213613,0.474205,122.175745,3.904218,NaN
std,NaN,NaN,NaN,NaN,22.283855,1.064131e+05,NaN,0.173409,0.251053,3.560147,5.011422,0.480620,0.105762,0.331906,0.309217,0.190462,0.259204,29.972954,0.432117,NaN
min,NaN,NaN,NaN,NaN,0.000000,8.586000e+03,NaN,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,NaN,NaN,NaN,NaN,17.000000,1.741840e+05,NaN,0.456000,0.473000,2.000000,-9.998000,0.000000,0.035900,0.016800,0.000000,0.098000,0.260000,99.296000,4.000000,NaN
50%,NaN,NaN,NaN,NaN,35.000000,2.130000e+05,NaN,0.580000,0.685000,5.000000,-6.997000,1.000000,0.048900,0.168000,0.000041,0.132000,0.464000,122.020000,4.000000,NaN
75%,NaN,NaN,NaN,NaN,50.000000,2.615880e+05,NaN,0.695000,0.854000,8.000000,-5.001000,1.000000,0.084500,0.596000,0.048700,0.273000,0.683000,140.074000,4.000000,NaN


There does not seem to be invalid data in the dataset. But one thing we should do to prepare the dataset for analysis later is to convert popularity column to be out of 1 instead of 100. Because that is what the other measures are out of (danceability, energy, speechiness, etc).

In [13]:
df['popularity'] = df['popularity'].div(100)

In [14]:
df["duration_mins"] = df["duration_ms"]/60000
df.drop(columns="duration_ms", inplace=True)

In [15]:
# check to see if things have been fixed
df.describe(include='all')

,track_id,artists,album_name,track_name,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,duration_mins
count,113549,113549,113549,113549,113549.000000,113549,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549,113549.000000
unique,89740,31437,46589,73608,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,114,NaN
top,6S3JlDAGk3uu3NtZbPnuhS,The Beatles,Alternative Christmas 2022,Run Rudolph Run,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,acoustic,NaN
freq,9,279,195,151,NaN,103831,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000,NaN
mean,NaN,NaN,NaN,NaN,0.333244,NaN,0.567031,0.642091,5.309452,-8.243408,0.637866,0.084674,0.314064,0.155703,0.213613,0.474205,122.175745,3.904218,NaN,3.801356
std,NaN,NaN,NaN,NaN,0.222839,NaN,0.173409,0.251053,3.560147,5.011422,0.480620,0.105762,0.331906,0.309217,0.190462,0.259204,29.972954,0.432117,NaN,1.773552
min,NaN,NaN,NaN,NaN,0.000000,NaN,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,0.143100
25%,NaN,NaN,NaN,NaN,0.170000,NaN,0.456000,0.473000,2.000000,-9.998000,0.000000,0.035900,0.016800,0.000000,0.098000,0.260000,99.296000,4.000000,NaN,2.903067
50%,NaN,NaN,NaN,NaN,0.350000,NaN,0.580000,0.685000,5.000000,-6.997000,1.000000,0.048900,0.168000,0.000041,0.132000,0.464000,122.020000,4.000000,NaN,3.550000
75%,NaN,NaN,NaN,NaN,0.500000,NaN,0.695000,0.854000,8.000000,-5.001000,1.000000,0.084500,0.596000,0.048700,0.273000,0.683000,140.074000,4.000000,NaN,4.359800


In [16]:
df.head()

,track_id,artists,album_name,track_name,popularity,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,duration_mins
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,0.73,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic,3.844433
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,0.55,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic,2.493500
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,0.57,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic,3.513767
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,0.71,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic,3.365550
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,0.82,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic,3.314217


In [17]:
df.describe()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_mins
count,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000,113549.000000
mean,0.333244,0.567031,0.642091,5.309452,-8.243408,0.637866,0.084674,0.314064,0.155703,0.213613,0.474205,122.175745,3.904218,3.801356
std,0.222839,0.173409,0.251053,3.560147,5.011422,0.480620,0.105762,0.331906,0.309217,0.190462,0.259204,29.972954,0.432117,1.773552
min,0.000000,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.143100
25%,0.170000,0.456000,0.473000,2.000000,-9.998000,0.000000,0.035900,0.016800,0.000000,0.098000,0.260000,99.296000,4.000000,2.903067
50%,0.350000,0.580000,0.685000,5.000000,-6.997000,1.000000,0.048900,0.168000,0.000041,0.132000,0.464000,122.020000,4.000000,3.550000
75%,0.500000,0.695000,0.854000,8.000000,-5.001000,1.000000,0.084500,0.596000,0.048700,0.273000,0.683000,140.074000,4.000000,4.359800
max,1.000000,0.985000,1.000000,11.000000,4.532000,1.000000,0.965000,0.996000,1.000000,1.000000,0.995000,243.372000,5.000000,87.288250


## Encode and Transformation


Prepare our feature list, X

In [231]:
#X = df[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']]
X = df[['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'loudness',  'valence', 'key', 'duration_mins', 'tempo']]
y = df['popularity']

In [19]:
#explicit_encoded = pd.get_dummies(df['explicit'], prefix="Explicit", drop_first=True)

In [20]:
#explicit_encoded.head()

In [21]:
#X = X.join(explicit_encoded)

In [22]:
X.head()

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,loudness,valence
0,0.676,0.4610,0.1430,0.0322,0.000001,0.3580,-6.746,0.715
1,0.420,0.1660,0.0763,0.9240,0.000006,0.1010,-17.235,0.267
2,0.438,0.3590,0.0557,0.2100,0.000000,0.1170,-9.734,0.120
3,0.266,0.0596,0.0363,0.9050,0.000071,0.1320,-18.515,0.143
4,0.618,0.4430,0.0526,0.4690,0.000000,0.0829,-9.681,0.167


## Splitting the Data

In [232]:
X_train, X_test, y_train, y_test = train_test_split(X,              #the input features
                                                    y,              #the label
                                                    test_size=0.2,  #set aside 30% of the data as the test set
                                                    random_state=7, #reproduce the results
                                                    shuffle=True
                                                   )

In [228]:

def reset_test():
    X_train, X_test, y_train, y_test = train_test_split(X,              #the input features
                                                    y,              #the label
                                                    test_size=0.2,  #set aside 30% of the data as the test set
                                                    random_state=7, #reproduce the results
                                                    shuffle=True
                                                   )

In [215]:
X_train.describe()

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,loudness,valence,key,duration_mins,tempo
count,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000
mean,0.566482,0.642271,0.084760,0.313801,0.155885,0.214062,-8.241697,0.473752,5.307511,3.801076,122.223644
std,0.173576,0.251179,0.106089,0.332084,0.309329,0.190953,5.010492,0.259443,3.561344,1.711484,30.002792
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.143100,0.000000
25%,0.456000,0.473000,0.035900,0.016500,0.000000,0.098000,-9.997000,0.259000,2.000000,2.903100,99.391000
50%,0.580000,0.685000,0.049000,0.168000,0.000042,0.132000,-6.985000,0.462000,5.000000,3.550267,122.037000
75%,0.694000,0.854000,0.084500,0.596000,0.048850,0.275000,-4.999000,0.683000,8.000000,4.364667,140.103500
max,0.985000,1.000000,0.965000,0.996000,1.000000,1.000000,4.532000,0.995000,11.000000,79.817100,243.372000


In [233]:
ctr = ColumnTransformer([('minmax', MinMaxScaler(), ['tempo', 'duration_mins',]),
                        ('categorical', OneHotEncoder(), ['key']),
                        ('std', StandardScaler(), ['instrumentalness', 'liveness', 'speechiness'])],
                       remainder='passthrough')

In [234]:
train_enc = ctr.fit_transform(X_train)
encoded_column_feature_names = ctr.get_feature_names_out(X.columns)
X_train = pd.DataFrame(train_enc, columns=encoded_column_feature_names)
test_enc = ctr.fit_transform(X_test)
encoded_column_feature_names = ctr.get_feature_names_out(X.columns)
X_test = pd.DataFrame(test_enc, columns=encoded_column_feature_names)

In [235]:
X_train.describe()

,minmax__tempo,minmax__duration_mins,categorical__key_0,categorical__key_1,categorical__key_2,categorical__key_3,categorical__key_4,categorical__key_5,categorical__key_6,categorical__key_7,...,categorical__key_10,categorical__key_11,std__instrumentalness,std__liveness,std__speechiness,remainder__danceability,remainder__energy,remainder__acousticness,remainder__loudness,remainder__valence
count,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,...,90839.000000,90839.000000,9.083900e+04,9.083900e+04,9.083900e+04,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000
mean,0.502209,0.045912,0.115061,0.093990,0.101884,0.031704,0.079878,0.082079,0.068726,0.115765,...,0.065423,0.081617,-5.862590e-17,-8.068394e-17,-2.581260e-18,0.566482,0.642271,0.313801,-8.241697,0.473752
std,0.123280,0.021481,0.319097,0.291817,0.302497,0.175213,0.271105,0.274487,0.252989,0.319945,...,0.247273,0.273782,1.000006e+00,1.000006e+00,1.000006e+00,0.173576,0.251179,0.332084,5.010492,0.259443
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-5.039496e-01,-1.121029e+00,-7.989604e-01,0.000000,0.000000,0.000000,-49.531000,0.000000
25%,0.408391,0.034641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-5.039496e-01,-6.078100e-01,-4.605619e-01,0.456000,0.473000,0.016500,-9.997000,0.259000
50%,0.501442,0.042764,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-5.038148e-01,-4.297544e-01,-3.370794e-01,0.580000,0.685000,0.168000,-6.985000,0.462000
75%,0.575676,0.052985,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,-3.460264e-01,3.191268e-01,-2.451395e-03,0.694000,0.854000,0.596000,-4.999000,0.683000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,2.728870e+00,4.115902e+00,8.297266e+00,0.985000,1.000000,0.996000,4.532000,0.995000


In [236]:
X_test.describe()

,minmax__tempo,minmax__duration_mins,categorical__key_0,categorical__key_1,categorical__key_2,categorical__key_3,categorical__key_4,categorical__key_5,categorical__key_6,categorical__key_7,...,categorical__key_10,categorical__key_11,std__instrumentalness,std__liveness,std__speechiness,remainder__danceability,remainder__energy,remainder__acousticness,remainder__loudness,remainder__valence
count,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,...,22710.00000,22710.000000,2.271000e+04,2.271000e+04,2.271000e+04,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000
mean,0.569955,0.039110,0.112461,0.096962,0.102994,0.029414,0.075517,0.082299,0.072567,0.118142,...,0.06517,0.080889,1.251506e-17,-3.566793e-17,-1.267150e-17,0.569229,0.641371,0.315114,-8.250253,0.476017
std,0.139485,0.023051,0.315940,0.295912,0.303958,0.168969,0.264230,0.274825,0.259430,0.322783,...,0.24683,0.272671,1.000022e+00,1.000022e+00,1.000022e+00,0.172724,0.250553,0.331200,5.015243,0.258243
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,-5.019094e-01,-1.123831e+00,-8.074369e-01,0.000000,0.000019,0.000000,-46.591000,0.000000
25%,0.462573,0.028754,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,-5.019094e-01,-6.049301e-01,-4.646687e-01,0.459000,0.471000,0.018000,-9.999750,0.262250
50%,0.569915,0.036189,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,-5.017811e-01,-4.287798e-01,-3.411573e-01,0.582000,0.685000,0.171000,-7.037000,0.470500
75%,0.654243,0.045321,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,-3.491242e-01,3.140229e-01,2.568344e-03,0.697000,0.852750,0.595000,-5.011000,0.684000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.00000,1.000000,2.736780e+00,4.139457e+00,8.393687e+00,0.975000,1.000000,0.996000,1.864000,0.993000


In [132]:
one = OneHotEncoder()
copye = X_train[['key']].copy()
encoded_column = one.fit_transform(copye)
encoded_column_feature_names = one.get_feature_names_out(['key'])
df_encoded_column = pd.DataFrame(encoded_column.toarray(), columns=encoded_column_feature_names)
df_encoded_column.describe()

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
count,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000
mean,0.115061,0.093990,0.101884,0.031704,0.079878,0.082079,0.068726,0.115765,0.064851,0.099021,0.065423,0.081617
std,0.319097,0.291817,0.302497,0.175213,0.271105,0.274487,0.252989,0.319945,0.246264,0.298692,0.247273,0.273782
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Scaling/Normalizing Training Set 

In [39]:
reset_test()

In [133]:
X_train

,danceability,energy,speechiness,acousticness,instrumentalness,liveness,loudness,valence,key
109325,0.643,0.968,0.0589,0.029000,0.042600,0.0994,-5.323,0.2300,4
11902,0.328,0.223,0.0366,0.555000,0.964000,0.0822,-16.202,0.0379,10
54180,0.727,0.978,0.0437,0.000694,0.781000,0.1150,-7.284,0.3330,1
41290,0.347,0.964,0.0668,0.000230,0.343000,0.1160,-6.264,0.1430,5
99711,0.481,0.590,0.0903,0.707000,0.000370,0.1410,-7.145,0.4540,2
...,...,...,...,...,...,...,...,...,...
104446,0.461,0.801,0.0298,0.001150,0.000036,0.2740,-11.133,0.9630,7
10756,0.650,0.759,0.0304,0.008980,0.002400,0.0858,-5.095,0.4250,6
49898,0.823,0.837,0.1500,0.210000,0.000137,0.0691,-5.261,0.3310,10
58810,0.362,0.970,0.2130,0.000059,0.030500,0.0489,-5.797,0.2970,8


In [134]:
standard_columns = ['danceability', 'energy', 'acousticness', 'instrumentalness', 'valence']
min_max_columns = ['speechiness', 'liveness']
ct = ColumnTransformer(transformers=[('scaler', StandardScaler(), standard_columns), 
                                     ('minmax', MinMaxScaler(), min_max_columns),])
                       #('onehot', OneHotEncoder(), ['key'])])


In [135]:
ct.fit(X_train)

ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                 ['danceability', 'energy', 'acousticness',
                                  'instrumentalness', 'valence']),
                                ('minmax', MinMaxScaler(),
                                 ['speechiness', 'liveness'])])

In [136]:
X_scaled = ct.transform(X_train)

In [137]:
# Get the names of the scaled columns
columns_scaled = standard_columns.copy()
min_max_scaled = min_max_columns.copy()

# Get the names of the encoded columns
# columns_encoded = []
# for transformer_name, transformer, features in ct.transformers:
#     if transformer_name == 'onehot':
#         print(features)
#         print(transformer_name)
#         columns_encoded += transformer.get_feature_names_out('key').tolist()

# Concatenate the scaled and encoded columns into a new DataFrame
df_scaled_encoded = pd.DataFrame(X_scaled, columns=columns_scaled + min_max_scaled)


In [138]:
X_train = pd.DataFrame(X_scaled, columns=standard_columns + min_max_columns)

In [141]:
X_train = pd.concat([X_train, df_encoded_column],axis=1)

In [142]:
X_train.describe()

,danceability,energy,acousticness,instrumentalness,valence,speechiness,liveness,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
count,9.083900e+04,9.083900e+04,9.083900e+04,9.083900e+04,9.083900e+04,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000,90839.000000
mean,2.175298e-16,8.254166e-17,-1.533112e-17,-5.862590e-17,-6.527460e-17,0.087834,0.214062,0.115061,0.093990,0.101884,0.031704,0.079878,0.082079,0.068726,0.115765,0.064851,0.099021,0.065423,0.081617
std,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,1.000006e+00,0.109936,0.190953,0.319097,0.291817,0.302497,0.175213,0.271105,0.274487,0.252989,0.319945,0.246264,0.298692,0.247273,0.273782
min,-3.263605e+00,-2.557035e+00,-9.449499e-01,-5.039496e-01,-1.826049e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,-6.365056e-01,-6.739082e-01,-8.952634e-01,-5.039496e-01,-8.277495e-01,0.037202,0.098000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.788097e-02,1.701150e-01,-4.390512e-01,-5.038148e-01,-4.529832e-02,0.050777,0.132000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,7.346557e-01,8.429447e-01,8.497859e-01,-3.460264e-01,8.065328e-01,0.087565,0.275000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.411160e+00,1.424206e+00,2.054307e+00,2.728870e+00,2.009118e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
train_std_scaler = StandardScaler()
X_train = train_std_scaler.fit_transform(X_train)

X_train = pd.DataFrame(X_train, columns=['danceability', 'energy', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'time_signature', 'Explicit_True'])

In [ ]:
X_train.describe()

## Scaling/Normalizing Test Set 

In [143]:
one = OneHotEncoder()
copye = X_test[['key']].copy()
encoded_column = one.fit_transform(copye)
encoded_column_feature_names = one.get_feature_names_out(['key'])
df_encoded_column = pd.DataFrame(encoded_column.toarray(), columns=encoded_column_feature_names)
df_encoded_column.describe()

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
count,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.00000,22710.000000
mean,0.112461,0.096962,0.102994,0.029414,0.075517,0.082299,0.072567,0.118142,0.063672,0.099912,0.06517,0.080889
std,0.315940,0.295912,0.303958,0.168969,0.264230,0.274825,0.259430,0.322783,0.244174,0.299889,0.24683,0.272671
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [144]:
standard_columns = ['danceability', 'energy', 'acousticness', 'instrumentalness', 'valence']
min_max_columns = ['speechiness', 'liveness']
ct = ColumnTransformer(transformers=[('scaler', StandardScaler(), standard_columns), 
                                     ('minmax', MinMaxScaler(), min_max_columns),])
                       #('onehot', OneHotEncoder(), ['key'])])


In [145]:
ct.fit(X_test)

ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                 ['danceability', 'energy', 'acousticness',
                                  'instrumentalness', 'valence']),
                                ('minmax', MinMaxScaler(),
                                 ['speechiness', 'liveness'])])

In [146]:
X_scaled = ct.transform(X_test)

In [147]:
# Get the names of the scaled columns
columns_scaled = standard_columns.copy()
min_max_scaled = min_max_columns.copy()

# Get the names of the encoded columns
# columns_encoded = []
# for transformer_name, transformer, features in ct.transformers:
#     if transformer_name == 'onehot':
#         print(transformer_name)
#         columns_encoded += transformer.get_feature_names_out(['key']).tolist()

# Concatenate the scaled and encoded columns into a new DataFrame
df_scaled_encoded = pd.DataFrame(X_scaled, columns=columns_scaled + min_max_scaled)


In [148]:
X_test = pd.DataFrame(X_scaled, columns=standard_columns + min_max_columns)

In [149]:
X_test = pd.concat([X_test, df_encoded_column],axis=1)

In [150]:
X_test.describe()

,danceability,energy,acousticness,instrumentalness,valence,speechiness,liveness,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,key_10,key_11
count,2.271000e+04,2.271000e+04,2.271000e+04,2.271000e+04,2.271000e+04,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.000000,22710.00000,22710.000000
mean,1.523709e-16,8.400736e-17,1.451747e-16,1.251506e-17,-2.258969e-16,0.087754,0.213523,0.112461,0.096962,0.102994,0.029414,0.075517,0.082299,0.072567,0.118142,0.063672,0.099912,0.06517,0.080889
std,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,1.000022e+00,0.108685,0.189999,0.315940,0.295912,0.303958,0.168969,0.264230,0.274825,0.259430,0.322783,0.244174,0.299889,0.24683,0.272671
min,-3.295665e+00,-2.559803e+00,-9.514528e-01,-5.019094e-01,-1.843330e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
25%,-6.381945e-01,-6.799950e-01,-8.971037e-01,-5.019094e-01,-8.277922e-01,0.037253,0.098589,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
50%,7.393806e-02,1.741355e-01,-4.351365e-01,-5.017811e-01,-2.136412e-02,0.050676,0.132056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
75%,7.397530e-01,8.436701e-01,8.450864e-01,-3.491242e-01,8.053941e-01,0.088033,0.273185,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
max,2.349288e+00,1.431384e+00,2.055863e+00,2.736780e+00,2.001967e+00,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000


In [ ]:
scaler = MinMaxScaler()
X_test = scaler.fit_transform(X_test[['danceability', 'energy', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence',]])

In [ ]:
test_std_scaler = StandardScaler()
X_test = test_std_scaler.fit_transform(X_test.to_numpy())

X_test = pd.DataFrame(X_test, columns=['danceability', 'energy','speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', ])

In [ ]:
# lambda numpy array transformed 

In [ ]:
X.describe()
# key, loudness, tempo, normalized

In [ ]:
from sklearn.preprocessing import FunctionTransformer
# x < 0.1 -> 1, x > 0.95 -> 3, otherwise avg -> 2

#speechiness, 

# loudness, minmax 
def transform_instrumentalness():
    

In [ ]:
X_train.head()

## Building the RandomForestRegressor

In [ ]:
X_train.describe()

In [ ]:
random_forest = RandomForestRegressor()
params = {'n_estimators': np.arange(60,90,10),
         #'max_depth':np.arange(10,20,2),
          "min_samples_split": np.arange(2,6,2),
         #'class_weight': np.arange(1,10,1),
         'max_features':[4,5,6]}
        
# Hyperparmeters problem
# Best possible config 
# cv 7/8
random_forest_grid = GridSearchCV(random_forest, params, cv=5, return_train_score=True)
random_forest_grid.fit(X_train, y_train)

In [ ]:
print(f'Best parameters were: {random_forest_grid.best_params_}')

print(f'Best model: {random_forest_grid.best_estimator_}')

#print(f'Best features: {random_forest_grid.best_features_}')

In [ ]:
print(f'Best score was: {random_forest_grid.best_score_}')

In [ ]:
print(f'Best estimator was: {random_forest_grid.best_estimator_}')

In [ ]:
y_pred = random_forest_grid.predict(X_test_preprocessed)


In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
random_forest_grid.score(X_test_preprocessed, y_test)

In [ ]:
reset()

In [269]:
model = RandomForestRegressor(n_estimators=100, max_features=10, min_samples_split=5, min_samples_leaf=2, max_depth=10,n_jobs=-1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

#run_model(model, "Random Forrest Regressor")
# using link:
# score : 0.26
# mse: 0.03
# r2: 0.269

In [270]:
model.score(X_test, y_test)

0.12081173522298538

In [271]:
from sklearn.metrics import accuracy_score, r2_score
mean_squared_error(y_test, y_pred)

0.04356248395621316

In [272]:
r2_score(y_test, y_pred)

0.12081173522298538

In [ ]:

cv_results = pd.DataFrame(random_forest_grid.cv_results_)
cv_results = cv_results.sort_values("mean_test_score", ascending=False)
cv_results[['mean_train_score', 'std_train_score', 'mean_test_score', 'std_test_score']].head()

In [ ]:
#plot predicted vs actual y values

In [ ]:
rf = RandomForestRegressor(n_estimators=10)
rf.fit(X_train, y_train)

In [ ]:
y_pred_normal = rf.predict(X_test)

In [ ]:
mean_squared_error(y_test, y_pred_normal)

In [ ]:
rf.score(X_test, y_test)

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(y_test, y_pred, c='crimson')
plt.yscale('log')
plt.xscale('log')

p1 = max(max(y_pred), max(y_test))
p2 = min(min(y_pred), min(y_test))
plt.plot([p1, p2], [p1, p2], 'b-')
plt.xlabel('True Values', fontsize=15)
plt.ylabel('Predictions', fontsize=15)
plt.axis('equal')
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(y_test, y_pred, edgecolors=(0, 0, 0))
ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
ax.set_xlabel('Measured')
ax.set_ylabel('Predicted')
plt.show()

In [ ]:
y_test.describe()

In [ ]:
y.describe()

In [ ]:
importances = model.feature_importances_ # get all of the feature importances
forest_importances = pd.Series(importances, index=X.columns) # create separate series
std = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
fig, ax = plt.subplots() # create figure
forest_importances.plot.bar(yerr=std, ax=ax)
ax.set_title("Feature importances using MDI")
ax.set_ylabel("Mean decrease in impurity")
fig.tight_layout()

In [ ]:
X_train.columns

In [ ]:
X_train_preprocessed

In [ ]:
importances = model.feature_importances_

# Sort feature importances in descending order
indices = np.argsort(importances)[::-1]

# Rearrange feature names so they match the sorted feature importances
names = [X_train.columns[i] for i in indices]

# Create plot
plt.figure()

# Create plot title
plt.title("Feature Importance")

# Add bars
plt.bar(range(X_train.shape[1]), importances[indices])

# Add feature names as x-axis labels
plt.xticks(range(X_train.shape[1]), names, rotation=90)

# Show plot
plt.show()